In [ ]:
from collections import deque

def levels_and_peaksDict(w, positions):
    '''
    A helper function that computes the levels of given positions in a Dyck word w.
    w: list. The input Dyck word.
    positions: set. The positions to compute levels for.
    Returns: list. The levels of the given positions.
    '''
    level = 0
    levels_list = []
    peaksDict = {}
    for i in range(len(w)):
        if w[i] == 1:
            level += 1
        else:
            level -= 1
        if i in positions:
            levels_list.append(level)
        if i < len(w) and w[i] == 1 and w[i+1] == 0:
            peaksDict.setdefault(level, deque([])).appendleft(i)
    return levels_list, peaksDict


def scaffolding(w):
    '''
    Returns the output of Zeta map via a "scaffolding" algorithm that only reads information from the right
    once levels are computed. We are obeying the south-west level convention.

    w: list. The input Dyck word.
    '''
    n = len(w)  # semi-length of w

    # 1. Extract the right steps's positions:
    Rs_pos = set(i for i in range(n) if w[i] == 0)

    # 2. Construct the levels for the rights:
    Rs_levels, peaksDict = levels_and_peaksDict(w,Rs_pos)   # levels is a helper function that returns the levels of given positions

    # 3. The scaffolding algorithm:
    out = []    # to return result
    deque = []  # a mostly sorted queue that stores tuples (level, position)
            # to be visited, where insertions might also be performed.
    cur_level = max(peaksDict)  # to keep track of which level we are iterating
    alive_agents = []   # to keep track of where we are on the scaffolding. Format: [pos, if_right]

    while len(out) < n:

        peaks = peaksDict.get(cur_level, [])
        if peaks:    # if there are peaks of this level
            deque += peaks    # append the ups before them to deque

        if alive_agents:
            deque += alive_agents  # adding valid agents to deque

        deque.sort(reverse=True)        # sort the same level step by their positions
                                        # this should be fast as it is merging two sorted lists

        out += [w[pos] for pos in deque]    # append this to output
        deque = []              # clear deque

        # now let our agents on the scaffoldings observe and move
        # move existing ones first
        if alive_agents:
            for i, agent_pos in enumerate(alive_agents):
                if agent_pos in Rs_pos:  # agent is moving right
                    agent_pos = agent_pos + 1
                    if agent_pos in Rs_pos and agent_pos < 2*n: # so it's valid
                        alive_agents[i] = agent_pos  # move agent
                    else:
                        alive_agents.pop(i) # agent dies
                else:   # moving left
                    agent_pos = agent_pos - 1
                    if agent_pos not in Rs_pos and agent_pos >= 0:
                        alive_agents[i] = agent_pos
                    else:
                        alive_agents.pop(i) # agent die

        # then we add two agents to the right and left of peak
        for peaks_pos in peaks:
            agent_pos = peaks_pos + 1
            if agent_pos in Rs_pos and agent_pos < 2*n:
                alive_agents.append(agent_pos)
            agent_pos = peaks_pos - 1
            if agent_pos not in Rs_pos and agent_pos >= 0:
                alive_agents.append(agent_pos)
        
        # sort alive agents by position. This again is merging two sorted lists
        alive_agents.sort(reverse=True)

        # update current level
        cur_level -= 1
    
    return out

input = [1,0,1,1,0,1,0,0,1,0]

scaffolding(input)

[1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0]